In [ ]:
import cv2
import numpy as np
import h5py
import tifffile as tiff


with h5py.File('nuclei_cell_type.h5', 'r') as file:
    ids = file['ids'][:]
    data = file['data'][:]
    data_dict = dict(zip(ids, data))

mask_image = tiff.imread('all_genes_sum.tif')
image = tiff.imread('epoch_1_step_60_connected.tif')

scale_factor = 4  


output_image = np.full((image.shape[0]*scale_factor, image.shape[1]*scale_factor, 3), 255, dtype=np.uint8)


fixed_colors = [
    (181, 187, 169), (124, 213, 247), (80, 116, 137), (95, 149, 85), (137, 119, 63),
    (128, 154, 245), (231, 88, 112), (115, 73, 58), (80, 85, 71), (236, 173, 87),
    (97, 159, 162), (208, 176, 167), (144, 138, 146), (119, 76, 133), (225, 131, 249)
]


y_indices, x_indices = np.nonzero(mask_image != 0)
for y, x in zip(y_indices, x_indices):
    scaled_y, scaled_x = y * scale_factor, x * scale_factor
    output_image[scaled_y:scaled_y+1, scaled_x:scaled_x+1] = (0, 0, 0)


for id, color_value in data_dict.items():
    mask = (image == id)
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    scaled_contours = [contour * scale_factor for contour in contours]
    color = fixed_colors[int(color_value) % len(fixed_colors)]
    cv2.drawContours(output_image, scaled_contours, -1, color, 2)


flipped_output_image = cv2.flip(output_image, 0)


color_bar_width = 50
color_bar_height = 300
patch_height = 20  
x_offset = flipped_output_image.shape[1] - color_bar_width - 40 
y_offset = 10  

color_bar_image = np.full((color_bar_height, color_bar_width, 3), 255, dtype=np.uint8)

for index, color in enumerate(fixed_colors):
    center_y = index * patch_height + patch_height // 2
    center_x = 10 
    cv2.circle(color_bar_image, (center_x, center_y), 5, color, -1)  
    cv2.putText(color_bar_image, str(index), (center_x + 15, center_y + 5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)

border_thickness = 2
cv2.rectangle(color_bar_image, (0, 0), (color_bar_width, color_bar_height), (0, 0, 0), thickness=border_thickness)


final_image = flipped_output_image.copy()
final_image[y_offset:y_offset + color_bar_height, x_offset:x_offset + color_bar_width] = color_bar_image


cv2.imwrite('final_image.png', final_image)


In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt


file_path = "/home/zhw/PythonProjects/BIDCell/all_genes_sum.tif"
with rasterio.open(file_path) as dataset:
    band1 = dataset.read(1)
    plt.figure(figsize=(10, 10))
    plt.imshow(band1, cmap='gray')
    plt.axis('off')  
  
    plt.savefig("/home/zhw/PythonProjects/BIDCell/transcript_image.png", bbox_inches='tight', pad_inches=0)
    plt.show()